In [1]:
from numpy import *
from itertools import combinations, permutations

In [2]:
sequence = sorted([
    (31, 'E'), (32, 'D'), (33, 'F'), (35, 'A'), (37, 'B'), (38, 'C'), (39, 'E'),
    (40, 'F'), (42, 'C'), (44, 'D'), (46, 'B'), (47, 'A'), (48, 'D'), (50, 'C'),
    (53, 'E'), (54, 'F'), (55, 'C'), (57, 'B'), (58, 'E'), (59, 'A'), (60, 'E'),
    (61, 'C'), (62, 'F'), (65, 'A'), (67, 'D'),
], key=lambda x:x[0])

In [3]:
def createC1(sequence, step):
    C1 = {}
    for event in sequence:
        mo = tuple((event[0], event[0] + step)) # calc minimal occurrence (mo)
        evt1 = tuple(event[1])
        if evt1 in C1: # if event exists in C1
            if mo not in C1[evt1]: # if mo not exists in event's set of mo
                C1[evt1].add(mo)
        else: C1[evt1] = set((mo,))
    return C1

In [4]:
C1 = createC1(sequence, 1)
C1

{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
 ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
 ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
 ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
 ('B',): {(37, 38), (46, 47), (57, 58)},
 ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}}

In [5]:
def createLargeEpisodeSet(Ck, min_fr):
    Lk = Ck.copy()
    for episode in Ck:
        if len(Ck[episode]) < min_fr: # if number of mo < min_fr
            del Lk[episode]
    return Lk

In [6]:
L1 = createLargeEpisodeSet(C1, 2)
L1

{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
 ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
 ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
 ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
 ('B',): {(37, 38), (46, 47), (57, 58)},
 ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}}

In [7]:
def checkSubsetFrequency(candidate, Lk, k):
    if k > 1:    
        subsets = list(combinations(candidate, k))
    else:
        return True
    for elem in subsets:
        if not elem in Lk: #elem is tuple
            return False
    return True

In [8]:
def createCandidates(Lk, L1, k, max_width):
    # create candidates list
    candidatesK = [] #candidates list
    generatedCans = [] 
    lkKeys = sorted(set([item for t in Lk for item in t])) # get all episodes and sort / Lk:get all keys
    generatedCans = list(combinations(lkKeys, k))
    for can in generatedCans:
        if checkSubsetFrequency(can, Lk, k-1): #if all subsets of can is frequent / Lk:get all keys
            candidatesK.append(can)
    # create minimal occurrence of candidates
    if len(candidatesK) == 0: # if candidates list is null
        return dict()
    else:
        Ck = {}
        if k == 2: #candidate 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[tuple(can[0])]:
                    for mo1 in L1[tuple(can[1])]:
                        tsmin = min(mok[0], mo1[0])
                        temax = max(mok[1], mo1[1])
                        # temax - tsmin <= max_width
                        if temax - tsmin <= max_width:
                            Ck[can].add(tuple((tsmin, temax)))
        else: #candidate > 2
            for can in candidatesK: # can is tuple
                Ck[can] = set()
                for mok in Lk[can[:-1]]:
                    for mo1 in L1[tuple(can[-1])]:
                        tsmin = min(mok[0], mo1[0])
                        temax = max(mok[1], mo1[1])
                        # temax - tsmin <= max_width
                        if temax - tsmin <= max_width:
                            Ck[can].add(tuple((tsmin, temax)))
        # recheck minimal occurrence is minimal
        for episode, setOfMOs in Ck.items():
            rem = set() #remove set
            for mo1 in setOfMOs:
                for mo2 in setOfMOs.difference(mo1):
                    # mo1[0] is mo1.ts, mo2[0] is mo2.ts, mo1[1] is mo1.te, mo2[1] is mo2.te
                    if (mo1[0]==mo2[0] and mo1[1] < mo2[1]) or (mo1[0] > mo2[0] and mo1[1] == mo2[1]) or (mo1[0] > mo2[0] and mo1[1] < mo2[1]):
                        rem.add(mo2)
            Ck[episode] = setOfMOs.difference(rem)
        return Ck

In [9]:
C2 = createCandidates(L1, L1, 2, 5)
C2

{('A', 'B'): {(35, 38), (46, 48), (57, 60)},
 ('A', 'C'): {(35, 39), (47, 51), (55, 60), (59, 62), (61, 66)},
 ('A', 'D'): {(32, 36), (44, 48), (47, 49), (65, 68)},
 ('A', 'E'): {(31, 36), (35, 40), (58, 60), (59, 61)},
 ('A', 'F'): {(33, 36), (59, 63), (62, 66)},
 ('B', 'C'): {(37, 39), (42, 47), (46, 51), (55, 58), (57, 62)},
 ('B', 'D'): {(44, 47), (46, 49)},
 ('B', 'E'): {(37, 40), (53, 58), (57, 59)},
 ('B', 'F'): {(33, 38), (37, 41), (54, 58)},
 ('C', 'D'): {(42, 45), (48, 51)},
 ('C', 'E'): {(38, 40), (39, 43), (50, 54), (53, 56), (55, 59), (60, 62)},
 ('C', 'F'): {(38, 41), (40, 43), (50, 55), (54, 56), (61, 63)},
 ('D', 'E'): {(31, 33)},
 ('D', 'F'): {(32, 34), (40, 45)},
 ('E', 'F'): {(31, 34), (39, 41), (53, 55), (54, 59), (60, 63)}}

In [10]:
L2 = createLargeEpisodeSet(C2, 2)
L2

{('A', 'B'): {(35, 38), (46, 48), (57, 60)},
 ('A', 'C'): {(35, 39), (47, 51), (55, 60), (59, 62), (61, 66)},
 ('A', 'D'): {(32, 36), (44, 48), (47, 49), (65, 68)},
 ('A', 'E'): {(31, 36), (35, 40), (58, 60), (59, 61)},
 ('A', 'F'): {(33, 36), (59, 63), (62, 66)},
 ('B', 'C'): {(37, 39), (42, 47), (46, 51), (55, 58), (57, 62)},
 ('B', 'D'): {(44, 47), (46, 49)},
 ('B', 'E'): {(37, 40), (53, 58), (57, 59)},
 ('B', 'F'): {(33, 38), (37, 41), (54, 58)},
 ('C', 'D'): {(42, 45), (48, 51)},
 ('C', 'E'): {(38, 40), (39, 43), (50, 54), (53, 56), (55, 59), (60, 62)},
 ('C', 'F'): {(38, 41), (40, 43), (50, 55), (54, 56), (61, 63)},
 ('D', 'F'): {(32, 34), (40, 45)},
 ('E', 'F'): {(31, 34), (39, 41), (53, 55), (54, 59), (60, 63)}}

In [11]:
C3 = createCandidates(L2, L1, 3, 5)
C3

{('A', 'B', 'C'): {(35, 39), (46, 51), (55, 60), (57, 62)},
 ('A', 'B', 'D'): {(44, 48), (46, 49)},
 ('A', 'B', 'E'): {(35, 40), (57, 60)},
 ('A', 'B', 'F'): {(33, 38)},
 ('A', 'C', 'D'): {(47, 51)},
 ('A', 'C', 'E'): {(35, 40), (55, 60), (59, 62)},
 ('A', 'C', 'F'): {(59, 63), (61, 66)},
 ('A', 'D', 'F'): {(32, 36)},
 ('A', 'E', 'F'): {(31, 36), (59, 63)},
 ('B', 'C', 'D'): {(42, 47), (46, 51)},
 ('B', 'C', 'E'): {(37, 40), (53, 58), (55, 59), (57, 62)},
 ('B', 'C', 'F'): {(37, 41), (54, 58)},
 ('B', 'D', 'F'): set(),
 ('B', 'E', 'F'): {(37, 41), (53, 58), (54, 59)},
 ('C', 'D', 'F'): {(40, 45)},
 ('C', 'E', 'F'): {(38, 41), (39, 43), (50, 55), (53, 56), (54, 59), (60, 63)}}

In [12]:
L3 = createLargeEpisodeSet(C3, 2)
L3

{('A', 'B', 'C'): {(35, 39), (46, 51), (55, 60), (57, 62)},
 ('A', 'B', 'D'): {(44, 48), (46, 49)},
 ('A', 'B', 'E'): {(35, 40), (57, 60)},
 ('A', 'C', 'E'): {(35, 40), (55, 60), (59, 62)},
 ('A', 'C', 'F'): {(59, 63), (61, 66)},
 ('A', 'E', 'F'): {(31, 36), (59, 63)},
 ('B', 'C', 'D'): {(42, 47), (46, 51)},
 ('B', 'C', 'E'): {(37, 40), (53, 58), (55, 59), (57, 62)},
 ('B', 'C', 'F'): {(37, 41), (54, 58)},
 ('B', 'E', 'F'): {(37, 41), (53, 58), (54, 59)},
 ('C', 'E', 'F'): {(38, 41), (39, 43), (50, 55), (53, 56), (54, 59), (60, 63)}}

In [13]:
C4 = createCandidates(L3, L1, 4, 5)
C4

{('A', 'B', 'C', 'E'): {(35, 40), (55, 60), (57, 62)},
 ('A', 'C', 'E', 'F'): {(59, 63)},
 ('B', 'C', 'E', 'F'): {(37, 41), (53, 58), (54, 59)}}

In [14]:
def MinEpi(sequence, max_width, step, minFrequent):
    C1 = createC1(sequence, step)
    L1 = createLargeEpisodeSet(C1, minFrequent)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = createCandidates(L[k-2], L1, k, max_width)
        Lk = createLargeEpisodeSet(Ck, minFrequent)
        L.append(Lk)
        k += 1
    #remove empty last itemset from L
    if L[-1] == {}:
        L.pop()
    return L

In [15]:
L = MinEpi(sequence, max_width=5, step=1, minFrequent=2)
L

[{('E',): {(31, 32), (39, 40), (53, 54), (58, 59), (60, 61)},
  ('D',): {(32, 33), (44, 45), (48, 49), (67, 68)},
  ('F',): {(33, 34), (40, 41), (54, 55), (62, 63)},
  ('A',): {(35, 36), (47, 48), (59, 60), (65, 66)},
  ('B',): {(37, 38), (46, 47), (57, 58)},
  ('C',): {(38, 39), (42, 43), (50, 51), (55, 56), (61, 62)}},
 {('A', 'B'): {(35, 38), (46, 48), (57, 60)},
  ('A', 'C'): {(35, 39), (47, 51), (55, 60), (59, 62), (61, 66)},
  ('A', 'D'): {(32, 36), (44, 48), (47, 49), (65, 68)},
  ('A', 'E'): {(31, 36), (35, 40), (58, 60), (59, 61)},
  ('A', 'F'): {(33, 36), (59, 63), (62, 66)},
  ('B', 'C'): {(37, 39), (42, 47), (46, 51), (55, 58), (57, 62)},
  ('B', 'D'): {(44, 47), (46, 49)},
  ('B', 'E'): {(37, 40), (53, 58), (57, 59)},
  ('B', 'F'): {(33, 38), (37, 41), (54, 58)},
  ('C', 'D'): {(42, 45), (48, 51)},
  ('C', 'E'): {(38, 40), (39, 43), (50, 54), (53, 56), (55, 59), (60, 62)},
  ('C', 'F'): {(38, 41), (40, 43), (50, 55), (54, 56), (61, 63)},
  ('D', 'F'): {(32, 34), (40, 45)},

In [16]:
def isAlphaOccurInBeta(mob, mos_alpha): #mo_beta include (mo_beta_ts) and (mo_beta_ts + width2)
    for moa in mos_alpha:
        if mob[0] <= moa[0] and moa[1] <= mob[1]:
            return True
    return False

In [17]:
def calcConfidence(mos_beta, wid1, mos_alpha, wid2):
    denominator = 0.0
    numerator = 0
    for mob in mos_beta:
        #te - ts <= width1
        if mob[1] - mob[0] <= wid1:
            denominator += 1
            #number of mo_beta [mob.ts, mob.ts + wid2) contains mo_alpha is numerator
            if isAlphaOccurInBeta((mob[0], mob[0] + wid2), mos_alpha):
                numerator += 1
    if denominator == 0:
        return 0
    else:  
        return numerator/denominator

In [18]:
def calcSupport(mos_alpha, wid2):
    support = 0
    for moa in mos_alpha:
        #te - ts <= width2
        if moa[1] - moa[0] <= wid2:
            support += 1
    return support

In [19]:
def generateRules(L, max_width, step, minConf):
    #calculate time bound W
    time_bound = list(range(step, max_width, step))
    time_bound.append(max_width)
    #generate all rules
    for i in range(1, len(L)): #only get the sets with two or more items
        for episode in L[i]: #for each episode in a level
            for j in range(1, i+1): # i+1 equal to length of an episode, so j is the length of lhs
                lhsList = list(combinations(episode, j))
                for lhs in lhsList: #each left-hand-side
                    for rightWid in time_bound: #each time bound of rhs
                        for leftWid in time_bound: #each time bound of lhs
                            #index of L is the episode length-1
                            conf = calcConfidence(L[j-1][lhs], leftWid, L[i][episode], rightWid)
                            if conf >= minConf:
                                print(list(lhs), " [", leftWid, "] ==> ", list(episode), " [", rightWid, "] [", calcSupport(L[i][episode], rightWid), ", ", conf,"]", sep="")

In [20]:
generateRules(L, 5, 1, 0.9)

['B'] [1] ==> ['B', 'C'] [5] [5, 1.0]
['B'] [2] ==> ['B', 'C'] [5] [5, 1.0]
['B'] [3] ==> ['B', 'C'] [5] [5, 1.0]
['B'] [4] ==> ['B', 'C'] [5] [5, 1.0]
['B'] [5] ==> ['B', 'C'] [5] [5, 1.0]
['E'] [1] ==> ['E', 'F'] [5] [5, 1.0]
['E'] [2] ==> ['E', 'F'] [5] [5, 1.0]
['E'] [3] ==> ['E', 'F'] [5] [5, 1.0]
['E'] [4] ==> ['E', 'F'] [5] [5, 1.0]
['E'] [5] ==> ['E', 'F'] [5] [5, 1.0]
['A', 'B'] [2] ==> ['A', 'B', 'C'] [5] [4, 1.0]
['A', 'B'] [3] ==> ['A', 'B', 'C'] [5] [4, 1.0]
['A', 'B'] [4] ==> ['A', 'B', 'C'] [5] [4, 1.0]
['A', 'B'] [5] ==> ['A', 'B', 'C'] [5] [4, 1.0]
['A', 'B'] [2] ==> ['A', 'B', 'D'] [3] [1, 1.0]
['A', 'B'] [2] ==> ['A', 'B', 'D'] [4] [2, 1.0]
['A', 'B'] [2] ==> ['A', 'B', 'D'] [5] [2, 1.0]
['B', 'D'] [3] ==> ['A', 'B', 'D'] [4] [2, 1.0]
['B', 'D'] [4] ==> ['A', 'B', 'D'] [4] [2, 1.0]
['B', 'D'] [5] ==> ['A', 'B', 'D'] [4] [2, 1.0]
['B', 'D'] [3] ==> ['A', 'B', 'D'] [5] [2, 1.0]
['B', 'D'] [4] ==> ['A', 'B', 'D'] [5] [2, 1.0]
['B', 'D'] [5] ==> ['A', 'B', 'D'] [5] [2, 1